## Libraries (local machine)

In [6]:
from dotenv import load_dotenv
from os import getenv
import numpy as np
import pandas as pd

load_dotenv()

True

## Libraries (Google Colab)

In [ ]:
from google.colab import userdata
import numpy as np
import pandas as pd

def getenv(secretName: str, default_value):
  try:
    return userdata.get(secretName)
  except:
    return default_value

## Classes definition

In [ ]:
#from typing import Any, NewType

class Part_Number:
    def __init__(self, pn: str, complexity: str):
        self.pn = pn
        self.complexity = complexity

    def __str__(self):
        return self.pn

class ECN:
    def __init__(self, ecn_id: str, pn_list: list[Part_Number]):
        self.ecn_id = ecn_id
        self.items = pn_list
        self.rfqs = []

    def __str__(self):
        return self.ecn_id
    
class Quote:
    def __init__(self, ecn: ECN, supplier: object):
        self.ecn = ecn
        self.supplier = supplier

        self.supplier.rfqs.append(self)
        self.ecn.rfqs.append(self)

class Supplier:
    def __init__(self, id: str | int, name: str, price_profile: str = "regular", quotation_profile: str = "regular", sample_delivery_punctuality_profile: str = "regular"):
        self.id = self.__check_id(id)
        self.name = name
        self.rfqs = []

        price_profile_map = { # This is a factor to multiply; average and standard deviation
          "low": (0.85, 0.85),
          "regular": (1, 1),
          "high": (1.2, 1.1)
        }

        quotation_profile_map = { # This is a factor to multiply; average and standard deviation
          "low": (28.975, 25.1133753461483),
          "regular": (27.7241379310345, 21.5974276436511),
          "high": (24.9444444444444, 10.258266234788)
        }

        punctuality_profile_map = { # Probability
          "low": 0.19047619047619,
          "regular": 0.473684210526316,
          "high": 0.638888888888889
        }

        self.ETA_difference = {
            "punctual": (0.888888888888889, 1.01273936708367),
            "unpunctual": (4.24137931034483, 2.69463981708917)
        }

        µ_price_profile_factor, σ_price_profile_factor = price_profile_map[price_profile]

        self.µ_price_high_complexity = getenv("AVG_PRICE_LOW_COMPLEXITY") * µ_price_profile_factor
        self.σ_price_high_complexity = getenv("STDEV_PRICE_LOW_COMPLEXITY") * σ_price_profile_factor
        self.µ_price_medium_complexity = getenv("AVG_PRICE_MEDIUM_COMPLEXITY") * µ_price_profile_factor
        self.σ_price_medium_complexity = getenv("STDEV_PRICE_MEDIUM_COMPLEXITY") * σ_price_profile_factor
        self.µ_price_low_complexity = getenv("AVG_PRICE_HIGH_COMPLEXITY") * µ_price_profile_factor
        self.σ_price_low_complexity = getenv("STDEV_PRICE_HIGH_COMPLEXITY") * σ_price_profile_factor
        self.minimum_price = getenv("MINIMUM_PRICE") * µ_price_profile_factor

        self.µ_quotation_time = quotation_profile_map[quotation_profile]
        self.σ_quotation_time = quotation_profile_map[quotation_profile]
        
        self.µ_delivery_time = 34.6206896551724
        self.σ_delivery_time = 16.2802512871323

        self.punctual_p = punctuality_profile_map[sample_delivery_punctuality_profile]

    def __str__(self):
        return self.name

    def __check_id(self, id: str | int):
        if len(str(id)) < 8 or len(str(id)) > 8:
          raise Exception("Invalid supplier ID")
        else:
          return str(id)
        
    def quote(self, ecn: ECN):
        pass

In [8]:
class Environment:
  def __init__(self):
    self.suppliers = []
    self.ecns = []
    self.part_kinds = {
        "A": {
            "average": float(getenv("AVG_A_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_A_PART_KIND", 1)),
            "complexity": {"low": 0.6818181818182, "medium": 0.318181818181818, "high": 0},
            "parts": []
        },
        "B": {
            "average": float(getenv("AVG_B_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_B_PART_KIND", 1)),
            "complexity": {"low": 1/3, "medium": 2/3, "high": 0},
            "parts": []
        },
        "C": {
            "average": float(getenv("AVG_C_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_C_PART_KIND", 1)),
            "complexity": {"low": 1, "medium": 0, "high": 0},
            "parts": []
        },
        "D": {
            "average": float(getenv("AVG_D_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_D_PART_KIND", 1)),
            "complexity": {"low": 0.090909090909090909, "medium": 0.727272727272727, "high": 0.181818181818182},
            "parts": []
        },
        "E": {
            "average": float(getenv("AVG_E_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_E_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        },
        "F": {
            "average": float(getenv("AVG_F_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_F_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        },
        "G": {
            "average": float(getenv("AVG_G_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_G_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        }
    }

  def add_supplier(self, supplier: Supplier):
    self.suppliers.append(supplier)

  def gen_ecn(self, qty: int):
    for i in range(qty):
      ecn_part_numbers = []

      while len(ecn_part_numbers) == 0:
        for key in self.part_kinds.keys():
          kind_complexity_keys = list(self.part_kinds[key]["complexity"].keys())
          kind_complexity_probabilities = list(self.part_kinds[key]["complexity"].values())

          for j in range(max(int(np.random.normal(self.part_kinds[key]["average"], self.part_kinds[key]["stdev"])), 0)):
            category_part_number = len(self.part_kinds[key]["parts"]) + 1
            complexity = np.random.choice(kind_complexity_keys, p=kind_complexity_probabilities)

            part_number = Part_Number(pn=f"A0{key}{str(category_part_number).zfill(6)}", complexity=complexity)

            self.part_kinds[key]["parts"].append(part_number)
            ecn_part_numbers.append(part_number)

      ecn_number = len(self.ecns) + 1
      self.ecns.append(ECN(ecn_id=f"ECN{str(ecn_number).zfill(7)}", pn_list=ecn_part_numbers))

  def gen_initial_df(self):
    pass

In [ ]:
env = Environment()
env.gen_ecn(10)

for ecn in env.ecns:
    for item in ecn.items:
        print(item)